In [2]:
import mne

file_path = 'F:/CHB_MIT/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01_03.edf'
raw_data = mne.io.read_raw_edf(file_path, preload=True)

Extracting EDF parameters from F:\CHB_MIT\chb-mit-scalp-eeg-database-1.0.0\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\one\AppData\Local\Temp\ipykernel_21620\3564678192.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data = mne.io.read_raw_edf(file_path, preload=True)


# 查看信息

In [3]:
print(raw_data.info)

<Info | 8 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 13:43:04 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
 subject_info: <subject_info | his_id: Surrogate>
>


# 改变采样率

In [4]:
# 设置新的采样率为200Hz
new_sampling_rate = 200.0
raw_data.resample(new_sampling_rate)
print(raw_data.info)

<Info | 8 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: 2076-11-06 13:43:04 UTC
 nchan: 23
 projs: []
 sfreq: 200.0 Hz
 subject_info: <subject_info | his_id: Surrogate>
>


# 删除重复通道

In [5]:
print(raw_data.info['ch_names'])

['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']


In [6]:
raw_data.drop_channels(raw_data.info['ch_names'][-1])

<RawEDF | chb01_03.edf, 22 x 720000 (3600.0 s), ~120.9 MiB, data loaded>

# 滤波

In [7]:
# 对数据进行0.5-45Hz的带通滤波
low_freq = 0.5
high_freq = 45.0
raw_data.filter(low_freq, high_freq)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1321 samples (6.605 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


<RawEDF | chb01_03.edf, 22 x 720000 (3600.0 s), ~120.9 MiB, data loaded>

# Z-score 归一化

In [8]:
import numpy as np

In [9]:
# Z归一化
for channel in raw_data.ch_names:
    data, times = raw_data.get_data(picks=[channel], return_times=True)
    # 计算均值和标准差
    mean = np.mean(data)
    std = np.std(data)
    # 应用Z-score标准化
    normalized_data = (data - mean) / std
    # 将归一化后的数据放回原始对象中
    raw_data[channel] = normalized_data
print("Z-score 归一化完成")

Z-score 归一化完成


In [10]:
print(raw_data.get_data)

<bound method get_data of <RawEDF | chb01_03.edf, 22 x 720000 (3600.0 s), ~120.9 MiB, data loaded>>
